In [46]:
import pandas as pd
from math import sqrt
import numpy as np

In [2]:
nurse_log_df = pd.read_csv("../../output/nurseLog.csv")

In [3]:
display(nurse_log_df)

,time,nurse,x,y,event,action,patient
0,0.0,0,9.0,17.0,3,assign event,0.0
1,3.0,0,6.0,17.0,3,move to,0.0
2,6.0,0,6.0,14.0,3,move to,0.0
3,6.0,0,6.0,14.0,3,unassign event,0.0
4,6.0,0,6.0,14.0,1,assign event,0.0
5,8.0,0,6.0,12.0,1,move to,0.0
6,12.0,0,6.0,8.0,1,move to,0.0
7,14.0,0,6.0,6.0,1,move to,0.0
8,17.0,0,6.0,3.0,1,move to,0.0
9,20.0,0,3.0,3.0,1,move to,0.0


In [34]:
event_log_df = pd.read_csv("../../output/eventLog.csv")

In [35]:
display(event_log_df)

,time,event,action,patient,type,request_level
0,6.0,1,planned start,0.0,request,3.0
1,6.0,1,actual start,0.0,request,3.0
2,35.0,1,end,0.0,request,3.0
3,30.0,2,planned start,0.0,request,2.0
4,41.0,2,actual start,0.0,request,2.0
5,51.0,2,end,0.0,request,2.0
6,0.0,3,planned start,0.0,plan,NaN
7,0.0,3,actual start,0.0,plan,NaN
8,6.0,3,pause,0.0,plan,NaN
9,71.0,3,resume,0.0,plan,NaN


In [19]:
def row_diff(row_num: int, df: pd.DataFrame, col_name: str):
    #calculates difference between items in col_name in df on rows row_num and the one before it
    #the column must be of a numerical type
    if row_num < 1:
        raise Exception("Row_num must be at least 1 (second row)")
    start_row = df.loc[row_num - 1]
    end_row = df.loc[row_num]
    start_item = start_row[col_name]
    end_item = end_row[col_name]
    return end_item - start_item


In [20]:
def nurse_dst_walked(nurse_id, nurse_df):
    move_rows = nurse_df[(nurse_df['nurse'] == nurse_id) & (nurse_df['action'] == "move to")].index.tolist()
    total_dst = 0
    total_dst2 = 0
    for row in move_rows:
        x_dst = row_diff(row, nurse_df, "x")
        y_dst = row_diff(row, nurse_df, "y")
        distance = sqrt(x_dst ** 2 + y_dst ** 2)
        total_dst += distance
    
    return total_dst

In [22]:
def nurse_time_walked(nurse_id, nurse_df):
    move_rows = nurse_df[(nurse_df['nurse'] == nurse_id) & (nurse_df['action'] == "move to")].index.tolist()
    total_time = 0
    for row in move_rows:
        total_time += row_diff(row, nurse_df, "time")
    return total_time

In [23]:
def nurse_time_at_patient(nurse_id, patient_id, nurse_df: pd.DataFrame):
    #assuming the df is sorted by nurse_id first and time second
    time_rows = nurse_df[(nurse_df['nurse'] == nurse_id) & (nurse_df['patient'] == patient_id) & (nurse_df['action'] == "time at patient")].index.tolist()
    total_time = 0
    for row in time_rows:
        total_time += row_diff(row, nurse_df, "time")
    return total_time

print(nurse_time_at_patient(0, 0, nurse_log_df))

50.0


In [24]:
print(nurse_time_walked(0, nurse_log_df), nurse_dst_walked(0, nurse_log_df), nurse_time_walked(1, nurse_log_df), nurse_dst_walked(1, nurse_log_df))

40.0 40.0 40.0 40.0


In [42]:
display(event_log_df)

,time,event,action,patient,type,request_level
0,6.0,1,planned start,0.0,request,3.0
1,6.0,1,actual start,0.0,request,3.0
2,35.0,1,end,0.0,request,3.0
3,30.0,2,planned start,0.0,request,2.0
4,41.0,2,actual start,0.0,request,2.0
5,51.0,2,end,0.0,request,2.0
6,0.0,3,planned start,0.0,plan,NaN
7,0.0,3,actual start,0.0,plan,NaN
8,6.0,3,pause,0.0,plan,NaN
9,71.0,3,resume,0.0,plan,NaN


In [43]:
display(event_log_df)

,time,event,action,patient,type,request_level
0,6.0,1,planned start,0.0,request,3.0
1,6.0,1,actual start,0.0,request,3.0
2,35.0,1,end,0.0,request,3.0
3,30.0,2,planned start,0.0,request,2.0
4,41.0,2,actual start,0.0,request,2.0
5,51.0,2,end,0.0,request,2.0
6,0.0,3,planned start,0.0,plan,NaN
7,0.0,3,actual start,0.0,plan,NaN
8,6.0,3,pause,0.0,plan,NaN
9,71.0,3,resume,0.0,plan,NaN


In [47]:
def patient_time_waiting(patient_id, request_level=None):
    if request_level is None:
        request_events = event_log_df[(event_log_df['patient'] == patient_id) & (event_log_df['type'] == 'request')]
    else: 
        request_events = event_log_df[(event_log_df['patient'] == patient_id) & (event_log_df['type'] == 'request') & (event_log_df['request_level'] == request_level)]
    event_times = []
    total_time = 0
    for _, event in request_events.groupby('event'):
        planned_start = event[event['action'] == 'planned start']['time']
        end_time = event[event['action'] == 'end']['time']
        wait_time = end_time.values[0] - planned_start.values[0]
        total_time += wait_time
        event_times.append(wait_time)
    return total_time, event_times

print(patient_time_waiting(0))
print(np.average(patient_time_waiting(0)[1]))

(np.float64(181.0), [np.float64(131.0), np.float64(29.0), np.float64(21.0)])
60.333333333333336
